In [ ]:
import pandas as pd
import re
from datetime import datetime

In [ ]:
# weather data
station_path = "/Users/sztchloe/Desktop/Air-Quality-Data-main/data/Station Inventory EN.csv"
weather_path  = "/Users/sztchloe/Desktop/Air-Quality-Data-main/data/weather data.csv"
output_path   = "/Users/sztchloe/Desktop/Air-Quality-Data-main/data/weather_ON_2022_2024.csv"

# 1) station inventory has 3 disclaimer rows -> skip them
station_df = pd.read_csv(station_path, skiprows=3)
station_small = station_df[['Climate ID', 'Province']].copy()
station_small['Climate ID'] = station_small['Climate ID'].astype(str)

# 2) weather data
weather_df = pd.read_csv(weather_path, low_memory=False)
weather_df['Climate ID'] = weather_df['Climate ID'].astype(str)

# 3) merge province into weather
weather_with_prov = weather_df.merge(station_small, on='Climate ID', how='left')

# 4) keep Ontario only
weather_ON = weather_with_prov[weather_with_prov['Province'].str.upper() == 'ONTARIO'].copy()

# 5) filter dates
dates = pd.to_datetime(weather_ON['Date/Time'], errors='coerce')
weather_ON_filtered = weather_ON[dates.between('2022-04-26', '2024-09-26')].copy()

# 6) save
weather_ON_filtered.to_csv(output_path, index=False)
print("Saved to:", output_path)
print("Final rows:", len(weather_ON_filtered))

/var/folders/d9/bdtl0xmj1nj1ny0y3x_s_4g00000gn/T/ipykernel_39448/635392954.py:8: DtypeWarning: Columns (4,9,11,13,15,17,19,21,23,27,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  weather_df = pd.read_csv(weather_path)


Saved to: /Users/liyanwang/Desktop/weather_ON_2022_2024.csv


In [ ]:
# Traffic data

# 1) Read CSV
path = "/Users/sztchloe/Desktop/Air-Quality-Data-main/data/traffic data.csv"
df = pd.read_csv(path)

# 2) Split WKT -> longitude / latitude
wkt = df["WKT"].astype(str)  # safer than df.iloc[:,0]
coords = wkt.str.extract(r"POINT\s*\(\s*([-\d\.]+)\s+([-\d\.]+)\s*\)")
df["longitude"] = pd.to_numeric(coords[0], errors="coerce")
df["latitude"]  = pd.to_numeric(coords[1], errors="coerce")

# 3) Keep only date columns within 2022-04-26 to 2024-09-26
start = datetime(2022, 4, 26)
end   = datetime(2024, 9, 26)

date_cols = []
for c in df.columns:
    m = re.fullmatch(r"x(\d{4})_(\d{2})_(\d{2})", str(c))
    if m:
        d = datetime(int(m.group(1)), int(m.group(2)), int(m.group(3)))
        if start <= d <= end:
            date_cols.append(c)

non_date_cols = [c for c in df.columns if not re.fullmatch(r"x\d{4}_\d{2}_\d{2}", str(c))]

df_filtered = df[non_date_cols + date_cols].copy()

# 4) Save to an Excel file (must include filename)
out_path = "/Users/sztchloe/Desktop/Air-Quality-Data-main/data/traffic_ON_2022_2024.csv"
df_filtered.to_csv(out_path, index=False)


print("Saved to:", out_path)
print("Kept date columns:", len(date_cols))
print("Final shape:", df_filtered.shape)